In [ ]:
#!pip install imageio.v3
!pip install mplsoccer

# Import statements
import os
import numpy as np
import pandas as pd
from matplotlib import animation, patches, pyplot as plt
from mplsoccer import Pitch
import imageio.v3 as imageio

In [ ]:
# Params
GAME_DURATION_SECONDS = 5745.20
START_TIME = 0
FRAME_INCREMENT = 5
PITCH_WIDTH = 68
PITCH_LENGTH = 105
HALF_PITCH = True

In [ ]:
# Function to load data
def load_data(link):
    df = pd.read_csv(link, skiprows=2)
    df.sort_values('Time [s]', inplace=True)
    return df

def load_events(link):
    df = pd.read_csv(link)
    df.sort_values('End Frame', inplace=True)
    return df

# Load data
link_away = ('https://raw.githubusercontent.com/metrica-sports/sample-data/master/'
             'data/Sample_Game_1/Sample_Game_1_RawTrackingData_Away_Team.csv')
link_home = ('https://raw.githubusercontent.com/metrica-sports/sample-data/master/'
             'data/Sample_Game_1/Sample_Game_1_RawTrackingData_Home_Team.csv')
link_events = ('https://raw.githubusercontent.com/metrica-sports/sample-data/master/'
               'data/Sample_Game_1/Sample_Game_1_RawEventsData.csv')
df_away = load_data(link_away)
df_home = load_data(link_home)
df_events = load_events(link_events)

display(df_events)

,Team,Type,Subtype,Period,Start Frame,Start Time [s],End Frame,End Time [s],From,To,Start X,Start Y,End X,End Y
0,Away,SET PIECE,KICK OFF,1,1,0.04,0,0.00,Player19,NaN,NaN,NaN,NaN,NaN
1,Away,PASS,NaN,1,1,0.04,3,0.12,Player19,Player21,0.45,0.39,0.55,0.43
2,Away,PASS,NaN,1,3,0.12,17,0.68,Player21,Player15,0.55,0.43,0.58,0.21
3,Away,PASS,NaN,1,45,1.80,61,2.44,Player15,Player19,0.55,0.19,0.45,0.31
4,Away,PASS,NaN,1,77,3.08,96,3.84,Player19,Player21,0.45,0.32,0.49,0.47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1740,Home,PASS,NaN,2,143361,5734.44,143483,5739.32,Player12,Player13,0.60,0.33,0.19,0.95
1741,Home,PASS,NaN,2,143578,5743.12,143593,5743.72,Player13,Player4,0.09,0.88,0.14,0.69
1743,Away,RECOVERY,BLOCKED,2,143617,5744.68,143617,5744.68,Player16,NaN,0.05,0.62,NaN,NaN
1742,Home,BALL LOST,INTERCEPTION,2,143598,5743.92,143618,5744.72,Player4,NaN,0.13,0.69,0.07,0.61


In [ ]:
def cleanup_events(df):
    # Filter rows where Type is "SHOT"
    df_1 = df[df['Type'] == "SHOT"].copy()

    # Replace "Subtype" values using vectorized operation
    df_1['Goal?'] = df_1['Subtype'].str.contains("GOAL").map({True: "GOAL", False: "NO GOAL"})

    # Drop unnecessary columns
    cols_to_drop = ['Type', 'Period', 'Start Frame', 'Start Time [s]', 'End Time [s]',
                    'From', 'To', 'Start X', 'End X', 'Start Y', 'End Y', 'Subtype']
    df_2 = df_1.drop(columns=cols_to_drop)

    # Reset index
    df_2.reset_index(drop=True, inplace=True)

    return df_2


df_events = cleanup_events(df_events)
display(df_events)

,Team,End Frame,Goal?
0,Home,2309,GOAL
1,Home,5953,NO GOAL
2,Away,7789,NO GOAL
3,Home,9632,NO GOAL
4,Home,18301,NO GOAL
5,Away,19483,NO GOAL
6,Home,26636,NO GOAL
7,Home,31235,NO GOAL
8,Home,53707,NO GOAL
9,Home,53775,NO GOAL


In [ ]:
# Reset column names
def set_col_names(df):
    """Renames the columns to have x and y suffixes."""
    cols = list(np.repeat(df.columns[3::2], 2))
    cols = [col+'_x' if i % 2 == 0 else col+'_y' for i, col in enumerate(cols)]
    cols = np.concatenate([df.columns[:3], cols])
    df.columns = cols


set_col_names(df_away)
set_col_names(df_home)

# Split off ball data
df_ball = df_away[['Period', 'Frame', 'Time [s]', 'Ball_x', 'Ball_y']].copy()
df_home.drop(['Ball_x', 'Ball_y'], axis=1, inplace=True)
df_away.drop(['Ball_x', 'Ball_y'], axis=1, inplace=True)
df_ball.rename({'Ball_x': 'x', 'Ball_y': 'y'}, axis=1, inplace=True)


# Convert to long form
def to_long_form(df):
    """Pivots a dataframe from wide-form to long form."""
    df = pd.melt(df, id_vars=df.columns[:3], value_vars=df.columns[3:], var_name='player')
    df.loc[df.player.str.contains('_x'), 'coordinate'] = 'x'
    df.loc[df.player.str.contains('_y'), 'coordinate'] = 'y'
    df = df.dropna(axis=0, how='any')
    df['player'] = df.player.str[6:-2]
    df = (df.set_index(['Period', 'Frame', 'Time [s]', 'player', 'coordinate'])['value']
          .unstack()
          .reset_index()
          .rename_axis(None, axis=1))
    return df


df_away = to_long_form(df_away)
df_home = to_long_form(df_home)

# Drop off unnecessary columns
df_home.drop(['Period', 'Time [s]', 'player'], axis=1, inplace=True)
df_away.drop(['Period', 'Time [s]', 'player'], axis=1, inplace=True)
df_ball.drop(['Period', 'Time [s]'], axis=1, inplace=True)

#display(df_away)
#display(df_ball)


In [ ]:
amount_of_frames_per_event = 15


def get_frames_and_label_per_event(df, frame_amount):
    frames_per_event = {}

    for index, event in df.iterrows():
        last_frame = event['End Frame']
        start_frame = last_frame - (FRAME_INCREMENT * (frame_amount - 1))
        frames = list(range(start_frame, last_frame + 1, FRAME_INCREMENT))
        existing_frames = [frame for frame in frames if frame >= 0]

        frames_per_event.update({index: [existing_frames, event['Goal?']]})

    return frames_per_event


frames_and_label_per_event = get_frames_and_label_per_event(df_events, amount_of_frames_per_event)
print(frames_and_label_per_event)


{0: [[2239, 2244, 2249, 2254, 2259, 2264, 2269, 2274, 2279, 2284, 2289, 2294, 2299, 2304, 2309], 'GOAL'], 1: [[5883, 5888, 5893, 5898, 5903, 5908, 5913, 5918, 5923, 5928, 5933, 5938, 5943, 5948, 5953], 'NO GOAL'], 2: [[7719, 7724, 7729, 7734, 7739, 7744, 7749, 7754, 7759, 7764, 7769, 7774, 7779, 7784, 7789], 'NO GOAL'], 3: [[9562, 9567, 9572, 9577, 9582, 9587, 9592, 9597, 9602, 9607, 9612, 9617, 9622, 9627, 9632], 'NO GOAL'], 4: [[18231, 18236, 18241, 18246, 18251, 18256, 18261, 18266, 18271, 18276, 18281, 18286, 18291, 18296, 18301], 'NO GOAL'], 5: [[19413, 19418, 19423, 19428, 19433, 19438, 19443, 19448, 19453, 19458, 19463, 19468, 19473, 19478, 19483], 'NO GOAL'], 6: [[26566, 26571, 26576, 26581, 26586, 26591, 26596, 26601, 26606, 26611, 26616, 26621, 26626, 26631, 26636], 'NO GOAL'], 7: [[31165, 31170, 31175, 31180, 31185, 31190, 31195, 31200, 31205, 31210, 31215, 31220, 31225, 31230, 31235], 'NO GOAL'], 8: [[53637, 53642, 53647, 53652, 53657, 53662, 53667, 53672, 53677, 53682, 536

In [ ]:
def invert_coordinates(away_coordinates, home_coordinates, ball_coordinates):
    away_coordinates.loc[:, 'x'] = 1 - away_coordinates['x']
    home_coordinates.loc[:, 'x'] = 1 - home_coordinates['x']
    ball_coordinates.loc[:, 'x'] = 1 - ball_coordinates['x']

    return away_coordinates, home_coordinates, ball_coordinates

def segment_pitch(away_coordinates, home_coordinates, ball_coordinates):
    away_team_in_opposite_side_count = (away_coordinates['x'] < 0.5).sum()
    home_team_in_opposite_side_count = (home_coordinates['x'] > 0.5).sum()

    if away_team_in_opposite_side_count > home_team_in_opposite_side_count:
        print("Away team opposite side count:", away_team_in_opposite_side_count,\
              "; Home team opposite side count:", home_team_in_opposite_side_count)
        return invert_coordinates(away_coordinates, home_coordinates, ball_coordinates)

    return away_coordinates, home_coordinates, ball_coordinates

In [ ]:
list_of_frames_per_event = [frames_label_set[0] for frames_label_set in list(frames_and_label_per_event.values())]
frames = sum(list_of_frames_per_event, []) # Flatten list of lists

print(frames)

[2239, 2244, 2249, 2254, 2259, 2264, 2269, 2274, 2279, 2284, 2289, 2294, 2299, 2304, 2309, 5883, 5888, 5893, 5898, 5903, 5908, 5913, 5918, 5923, 5928, 5933, 5938, 5943, 5948, 5953, 7719, 7724, 7729, 7734, 7739, 7744, 7749, 7754, 7759, 7764, 7769, 7774, 7779, 7784, 7789, 9562, 9567, 9572, 9577, 9582, 9587, 9592, 9597, 9602, 9607, 9612, 9617, 9622, 9627, 9632, 18231, 18236, 18241, 18246, 18251, 18256, 18261, 18266, 18271, 18276, 18281, 18286, 18291, 18296, 18301, 19413, 19418, 19423, 19428, 19433, 19438, 19443, 19448, 19453, 19458, 19463, 19468, 19473, 19478, 19483, 26566, 26571, 26576, 26581, 26586, 26591, 26596, 26601, 26606, 26611, 26616, 26621, 26626, 26631, 26636, 31165, 31170, 31175, 31180, 31185, 31190, 31195, 31200, 31205, 31210, 31215, 31220, 31225, 31230, 31235, 53637, 53642, 53647, 53652, 53657, 53662, 53667, 53672, 53677, 53682, 53687, 53692, 53697, 53702, 53707, 53705, 53710, 53715, 53720, 53725, 53730, 53735, 53740, 53745, 53750, 53755, 53760, 53765, 53770, 53775, 57670, 57

In [ ]:
p = Pitch(pitch_type='metricasports', goal_type='line', half=HALF_PITCH,
          pitch_width=PITCH_WIDTH, pitch_length=PITCH_LENGTH,
          pad_left=None, pad_right=None, pad_bottom=None, pad_top=None,
          line_color=None, goal_alpha=0, line_alpha=0)

# Filter dataframes outside the loop
away_frames = {frame: df_away[df_away['Frame'] == frame] for frame in frames}
home_frames = {frame: df_home[df_home['Frame'] == frame] for frame in frames}
ball_frames = {frame: df_ball[df_ball['Frame'] == frame] for frame in frames}

filenames = []
for frame in frames:
    fig, ax = p.draw()

    away_coordinates = away_frames[frame]
    home_coordinates = home_frames[frame]
    ball_coordinates = ball_frames[frame]

    away_coordinates, home_coordinates, ball_coordinates = \
        segment_pitch(away_coordinates, home_coordinates, ball_coordinates)

    player_coordinates = pd.concat([away_coordinates, home_coordinates])

    # Plot Voronoi
    team1, team2 = p.voronoi(player_coordinates.x, player_coordinates.y,
                              [0]*11 + [1]*11)

    t1 = p.polygon(team1, ax=ax, fc='orange', ec='black', lw=0, alpha=1)
    t2 = p.polygon(team2, ax=ax, fc='dodgerblue', ec='black', lw=0, alpha=1)

    # Plot players
    #sc1 = p.scatter(away_coordinates.x, away_coordinates.y, c='dodgerblue', s=80, ec='k', ax=ax)
    #sc2 = p.scatter(home_coordinates.x, home_coordinates.y, c='orange', s=80, ec='k', ax=ax)

    # Plot ball
    #sc3 = p.scatter(ball_coordinates.x, ball_coordinates.y, c='white', s=30, ec='k', ax=ax)

    # Create file name and append it to a list
    filename = f'/content/drive/MyDrive/Master 1/Projet TER/Images/Metrica Sports/Game 1/Half Pitch Per Event/voronoi_event_{frame}.png'
    filenames.append(filename)

    # Save frame
    plt.savefig(filename, bbox_inches='tight', pad_inches=0)
    plt.close()




Away team opposite side count: 10 ; Home team opposite side count: 0
Away team opposite side count: 10 ; Home team opposite side count: 0
Away team opposite side count: 10 ; Home team opposite side count: 0
Away team opposite side count: 10 ; Home team opposite side count: 0
Away team opposite side count: 10 ; Home team opposite side count: 0
Away team opposite side count: 10 ; Home team opposite side count: 0
Away team opposite side count: 10 ; Home team opposite side count: 0
Away team opposite side count: 10 ; Home team opposite side count: 0
Away team opposite side count: 10 ; Home team opposite side count: 0
Away team opposite side count: 10 ; Home team opposite side count: 0
Away team opposite side count: 10 ; Home team opposite side count: 0
Away team opposite side count: 10 ; Home team opposite side count: 0
Away team opposite side count: 10 ; Home team opposite side count: 0
Away team opposite side count: 10 ; Home team opposite side count: 0
Away team opposite side count: 10 

In [ ]:
# Remove image files on drive
for filename in set(filenames):
    os.remove(filename)

In [ ]:
def generate_gif(filenames):
    # Generates a gif from the pre-generated temp images.
    # Build gif
    gif_name = f'Voronoi_{START_TIME}_{GAME_DURATION_SECONDS}_{FRAME_INCREMENT}.gif'
    images = []
    for filename in filenames:
       images.append(imageio.imread(filename))
    imageio.imwrite(gif_name, images, loop=0)

    # Remove temp image files
    for filename in set(filenames):
        os.remove(filename)


#generate_gif(filenames)

In [ ]:
# TESTS

print(away_coordinates)
away_coordinates.x = away_coordinates.x.map(lambda x: 1-x)
print(away_coordinates)
print(away_coordinates.x)
print(away_coordinates['x'])


       Frame        x        y
16555   1506  0.03437  0.28076
16556   1506  0.04796  0.38787
16557   1506  0.06597  0.47996
16558   1506  0.07699  0.33265
16559   1506  0.11313  0.41659
16560   1506  0.09116  0.49120
16561   1506  0.22736  0.54963
16562   1506  0.18101  0.58370
16563   1506  0.35980  0.55805
16564   1506  0.37221  0.43667
16565   1506  0.01806  0.44883
       Frame        x        y
16555   1506  0.96563  0.28076
16556   1506  0.95204  0.38787
16557   1506  0.93403  0.47996
16558   1506  0.92301  0.33265
16559   1506  0.88687  0.41659
16560   1506  0.90884  0.49120
16561   1506  0.77264  0.54963
16562   1506  0.81899  0.58370
16563   1506  0.64020  0.55805
16564   1506  0.62779  0.43667
16565   1506  0.98194  0.44883
16555    0.96563
16556    0.95204
16557    0.93403
16558    0.92301
16559    0.88687
16560    0.90884
16561    0.77264
16562    0.81899
16563    0.64020
16564    0.62779
16565    0.98194
Name: x, dtype: float64
16555    0.96563
16556    0.95204
16557    0.

<ipython-input-64-7d38383170bb>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  away_coordinates.x = away_coordinates.x.map(lambda x: 1-x)
